<a href="https://colab.research.google.com/github/harshitadd/CrossPriv/blob/master/PreVID_PySyft_Federated_Averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install syft==0.2.6 --quite 
#!pip install pydicom --quiet

In [26]:
import os 
from google.colab import drive
drive.mount('/content/drive')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time
import csv 
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker
import pydicom 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler 
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
def makeNonFedData():
  dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch/')
  dcm_data={}
  alpha = 1.5 
  beta = 0
  labels=[]
  pid=[]
  dicom=[]
  label=[]
  for file in dcm_path:
    name = '/content/drive/My Drive/Fed_Covid/minibatch/' + file
    temp = pydicom.dcmread(name)
    image = temp.pixel_array
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    adjusted = cv2.resize(image,(64,64))
    dcm_data[file]=adjusted  
  with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
    reader = csv.reader(file)
    for row in reader:
      labels.append(row)  
  scaler = MinMaxScaler()
  cid = 0
  for PID in labels:
    for key in dcm_data:
      if(key[:-4]==PID[0]):
        l=[]
        for val in dcm_data[key]:
          l.append(scaler.fit_transform(val))
        l = np.reshape(l,(3,64,64))
        dicom.append(l)
        label.append(int(PID[5]))
  
  return dicom,label
    

In [28]:
dicom, label = makeNonFedData()

In [29]:
class Arguments():
    def __init__(self):
        self.batch_size = 8
        self.test_batch_size = 4
        self.epochs = 1
args = Arguments()

In [30]:
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test
x_train= np.array(x_train)
y_train= np.array(y_train)
x_test= np.array(x_test)
y_test= np.array(y_test)

In [33]:
class data_maker(Dataset):
  def __init__(self, images, labels):
        self.data = images 
        self.targets = labels 

        self.to_torchtensor()
        
  def __len__(self):
      return len(self.data)

  def to_torchtensor(self):      
      self.data=torch.from_numpy(self.data)
      self.labels=torch.from_numpy(self.targets)
  
  def __getitem__(self, idx):
      sample=self.data[idx]
      target=self.targets[idx]
      return sample,target

In [34]:
hook = sy.TorchHook(torch)
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic = sy.VirtualWorker(hook, id="clinic")  

In [35]:
federated_train_data = data_maker(x_train,y_train).federate((hospital,clinic))

In [36]:
federated_train_loader = sy.FederatedDataLoader(federated_train_data,batch_size=args.batch_size)
test_data = data_maker(x_test,y_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size)

In [37]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(4,2,padding=0)
        self.fc1 = nn.Linear(256, 64*64)
        self.fc2 = nn.Linear(64*64, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,256)
     
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.log_softmax(x,dim=1)

In [38]:
hospital_model = Net().double()
clinic_model = Net().double()
hospital_optimizer = optim.Adam(hospital_model.parameters(), lr=0.0001)
clinic_optimizer = optim.Adam(clinic_model.parameters(), lr=0.0001)
models = [hospital_model, clinic_model]
optimizers = [hospital_optimizer, clinic_optimizer]
model = Net().double()

In [39]:
compute_nodes = [hospital, clinic]

In [40]:
def update(data, target, model, optimizer):
    loss = nn.CrossEntropyLoss()
    model.send(data.location) 
    optimizer.zero_grad()
    output = model(data)
    out = loss(output, target)
    out.backward()
    optimizer.step()
    return model

In [41]:
def train():
  for batch_idx, (data, target) in enumerate(federated_train_loader):
    for index in range(len(compute_nodes)):
      models[index] = update(data,target,models[index],optimizers[index]) 
    for model in models:
      model.get()
    return utils.federated_avg({"hospital": models[0],"clinic":models[1]}) #


In [44]:
def test(federated_model):
  federated_model.eval()
  test_loss = 0
  correct = 0
  loss = nn.CrossEntropyLoss()
  for data, target in test_loader:
        output = model(data)
        out = loss(output,target)
        test_loss +=out
        print('test loss of the iteration',test_loss)
        pred = output.argmax(1, keepdim=True)
        print('predicted val',pred)
        print('target val',target)
        correct += pred.eq(target.view_as(pred))

  test_loss /= len(test_loader.dataset)

  print('For Test Loss:' + str(test_loss))

In [ ]:
for epoch in range(5):
  start_time = time.time()
  print(f"Epoch Number {epoch}")
  federated_model = train()
  test(federated_model)
  total_time = time.time() - start_time 
  print('Comm time', round(total_time,2),'s\n')